# Neural Networks

### We are going to predict whether a customer will rent an entire apartment or a private room

In [1]:
import os
import glob
import math
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt

from IPython.display import display
from IPython.core.display import display, HTML

import keras
from keras.models import Sequential
from keras.layers import Dense

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

%matplotlib inline

# Loading csv files
path = os.getcwd()
all_files = glob.glob(os.path.join(path, "../../datasets/airbnb_survey/*2016*.csv"))
# all_files = glob.glob(os.path.join(path, "airbnb_survey/*.csv"))

# Concat all csv and include city names as column
df_each_file = (pd.read_csv(f).assign(city=os.path.basename(f).split('_')[2]) for f in all_files)
data = pd.concat(df_each_file, ignore_index=True)

Using TensorFlow backend.


In [2]:
# DATA CLEANING
# Initial analysis of data by dataframe.info() & dataframe.describe()

# Convert string date time to timestamp dtype
data.last_modified = data.last_modified.astype('datetime64[ns]')

# Dropping columns with all null values and unwanted columns
reject_cols = ['country', 'borough', 'minstay', 'bathrooms','location','name','room_id','survey_id','latitude','longitude']

for col in reject_cols:
    try:
        data.drop(col, axis=1, inplace=True)
    except ValueError:
        pass

#The property_value col is null for nearly 99% of records, so dropping it
try:
    data.drop(['property_type'], axis=1, inplace=True)
except ValueError:
    pass

#Droping rows as price is null and total null records is less than 1% of population size
if len(data[data.price.isnull()]) < len(data)*0.01:
    data.dropna(subset=['price'], inplace=True)

#Drop rows with null room_type as they won't be used in our analysis
#In this case there are only 2 rows 
data.dropna(subset=['room_type', 'bedrooms', 'accommodates'], inplace=True)


data.drop(data[data.price > 10000].index, inplace=True)

data.drop(data[data.room_type == 'Shared room'].index, inplace=True)
data.drop(['overall_satisfaction', 'host_id', 'last_modified', 'neighborhood'], axis=1, inplace=True)

In [3]:
data.head()

,room_type,reviews,accommodates,bedrooms,price,city
0,Entire home/apt,0,2,1.0,68.0,aarhus
1,Entire home/apt,1,2,0.0,61.0,aarhus
2,Entire home/apt,1,4,1.0,68.0,aarhus
3,Entire home/apt,5,4,2.0,87.0,aarhus
4,Private room,1,2,1.0,61.0,aarhus


In [4]:
# encode strings into numeric 
# Converts Entire home/apt=>0, Private room=>1, Shared room =>2

labelencoder_rt = LabelEncoder()
data.room_type = labelencoder_rt.fit_transform(data.room_type)

# Does the same for city column
labelencoder_city = LabelEncoder()
data.city = labelencoder_city.fit_transform(data.city)

y = data.iloc[:,0].values
X = data.iloc[:,1:].values

In [5]:
# Above, the values of city are replaced by 0 to 6 which implies that 2nd city is greater than 1st & 
# similarly 3rd is greater that 2nd. To solve this, we need to dummify the city column.
# This mthod is also known as folding or melting the column.

# pass the column number ([4] here) into the OneHotEncoder object
onehotencoder = OneHotEncoder(categorical_features = [4])
X = onehotencoder.fit_transform(X).toarray()
# X = X[:, 1:]

In [6]:
# Splitting the dataset into the Training set and Test set


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [7]:
try:
    del classifier
except NameError:
    pass

## Intialize neural network and add hidden layers

In [8]:
#Initializing Neural Network
classifier = Sequential()

1. output_dim: # of nodes to be added to the layer
2. init: initialization of Stochastic Gradient Decent.In Neural Network we need to assign weights to each mode which is nothing but importance of that node. At the time of initialization, weights should be close to 0 and we will randomly initialize weights using uniform function.
3. input_dim : # of input variables. Required only in first layer
4. Activation: activation function to be used - relu, sigmoid etc. Neuron applies activation function to weighted. The closer the activation function value to 1 the more activated is the neuron and more the neuron passes the signal.
    4.1 Here we are using rectifier(relu) function in our hidden layer and Sigmoid function in our output layer as we want binary result from output layer but if the number of categories in output layer is more than 2 then use SoftMax function.

In [9]:
# Adding the input layer and the first hidden layer
classifier.add(Dense(output_dim = 6, init = 'uniform', activation = 'relu', input_dim =10))

# Adding the second hidden layer
classifier.add(Dense(output_dim = 5, init = 'uniform', activation = 'relu', ))

# Adding the output layer
classifier.add(Dense(output_dim = 1, init = 'uniform', activation = 'sigmoid', ))

C:\Users\Niranjan\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=10, units=6, kernel_initializer="uniform")`
  
C:\Users\Niranjan\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=5, kernel_initializer="uniform")`
  """
C:\Users\Niranjan\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="sigmoid", units=1, kernel_initializer="uniform")`
  


1. optimizer; algo to be used to find optimal set of weights
2. This algorithm is Stochastic Gradient descent(SGD). Among several types of SGD algorithm the one which we will use is ‘Adam’. If you go in deeper detail of SGD, you will find that SGD depends on loss thus our second parameter is loss.
3. Since out dependent variable is binary, we will have to use logarithmic loss function called ‘binary_crossentropy’, if our dependent variable has more than 2 categories in output then use ‘categorical_crossentropy’.
4. We want to improve performance of our neural network based on accuracy so add metrics as accuracy

In [10]:
# Compiling Neural Network
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

1. Batch size : to specify the number of observation after which you want to update weight.
2. Epoch : total number of iterations.

In [11]:
# Fitting our model 
classifier.fit(X_train, y_train, batch_size = 20, nb_epoch = 15)

C:\Users\Niranjan\Anaconda3\lib\site-packages\keras\models.py:939: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/15
82901/82901 [==============================] - 5s 56us/step - loss: 0.3330 - acc: 0.8595
Epoch 2/15
82901/82901 [==============================] - 4s 50us/step - loss: 0.2664 - acc: 0.8947
Epoch 3/15
82901/82901 [==============================] - 5s 58us/step - loss: 0.2532 - acc: 0.8962
Epoch 4/15
82901/82901 [==============================] - 5s 58us/step - loss: 0.2477 - acc: 0.8970
Epoch 5/15
82901/82901 [==============================] - 5s 56us/step - loss: 0.2457 - acc: 0.8974
Epoch 6/15
82901/82901 [==============================] - 5s 56us/step - loss: 0.2449 - acc: 0.8973
Epoch 7/15
82901/82901 [==============================] - 5s 58us/step - loss: 0.2444 - acc: 0.8974
Epoch 8/15
82901/82901 [==============================] - 5s 56us/step - loss: 0.2437 - acc: 0.8977
Epoch 9/15
82901/82901 [==============================] - 4s 50us/step - loss: 0.2436 - acc: 0.8976
Epoch 10/15
82901/82901 [==============================] - 4s 53us/step - loss: 0.2433 - acc: 0.8977

In [12]:
# Predicting the Test set results

# The prediction result will give you probability of the customer leaving the company. 
# We will convert that probability into binary 0 and 1
y_pred = classifier.predict(X_test)



y_pred[:30]

y_pred_bin = (y_pred > 0.5)

In [13]:
# Creating the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred_bin)

In [14]:
cm

array([[15861,   817],
       [ 1293,  2755]])

In [15]:
accuracy = (cm[0][0] + cm[1][1]) / cm.sum()
accuracy

0.89819550323265462

In [16]:
scores =classifier.evaluate(X, y)

103627/103627 [==============================] - 4s 36us/step


In [17]:
print("\n%s: %.2f%%" % (classifier.metrics_names[1], scores[1]*100))
print("\n%s: %.2f" % (classifier.metrics_names[0], scores[0]))


acc: 79.90%

loss: 2.29


In [18]:
# Visualize he nodes 7 layers in neural network
from ann_visualizer.visualize import ann_viz;
os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin/'

ann_viz(classifier, title="Neural Netwok Viz")

In [19]:
# view viz
class PDF(object):
    def __init__(self, pdf, size=(200,200)):
        self.pdf = pdf
        self.size = size

    def _repr_html_(self):
        return '<iframe src={0} width={1[0]} height={1[1]}></iframe>'.format(self.pdf, self.size)

    def _repr_latex_(self):
        return r'\includegraphics[width=1.0\textwidth]{{{0}}}'.format(self.pdf)
    

PDF('network.gv.pdf',size=(600,500))